<a href="https://colab.research.google.com/github/rei-kun01/Simbolo_Webinar_Feb2025/blob/main/%5BSimbolo_webinar%5D_Transformer_simple_demonstration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Example sentence for translation
sentence = "Hello, how are you?"

# Tokenize the sentence
tokens = tokenizer.tokenize(sentence)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print("Original Sentence:", sentence)
print("Tokens:", tokens)
print("Token IDs:", token_ids)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Original Sentence: Hello, how are you?
Tokens: ['hello', ',', 'how', 'are', 'you', '?']
Token IDs: [7592, 1010, 2129, 2024, 2017, 1029]


In [ ]:
import torch
import torch.nn as nn

# Define embedding size
embedding_size = 8  # Keeping it small for demonstration

# Initialize embedding layer
embedding = nn.Embedding(num_embeddings=tokenizer.vocab_size, embedding_dim=embedding_size)

# Convert token IDs to a tensor
token_ids_tensor = torch.tensor(token_ids)

# Get embeddings
embeddings = embedding(token_ids_tensor)

print("Token IDs Tensor:", token_ids_tensor)
print("Embeddings:\n", embeddings)

Token IDs Tensor: tensor([7592, 1010, 2129, 2024, 2017, 1029])
Embeddings:
 tensor([[-1.2032,  1.6773,  1.3707,  1.1880,  0.8067,  0.0291, -0.2059, -1.3574],
        [ 0.5455, -1.8449,  0.4192, -0.4807,  0.2553, -1.2672,  0.4813,  1.6890],
        [-0.2986, -1.7394, -0.4345,  0.2187,  1.1005, -2.5130, -1.5706, -1.8841],
        [ 1.6727, -0.0950,  1.7118, -1.1858, -0.3260, -0.4416,  0.9235, -0.0940],
        [-0.4787, -0.4713,  2.8065,  1.0699, -0.2540,  0.3077, -1.8193,  1.6687],
        [ 0.2335,  0.2242, -0.0737, -0.3781,  0.0954,  1.5481,  1.4015,  0.4134]],
       grad_fn=<EmbeddingBackward0>)


In [ ]:
import math

def get_positional_encoding(seq_len, d_model):
    pe = torch.zeros(seq_len, d_model)
    for pos in range(seq_len):
        for i in range(0, d_model, 2):
            pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
            if i + 1 < d_model:
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
    return pe

# Get positional encoding
pos_encoding = get_positional_encoding(len(tokens), embedding_size)

print("Positional Encoding:\n", pos_encoding)

Positional Encoding:
 tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
          1.0000e+00,  0.0000e+00,  1.0000e+00],
        [ 8.4147e-01,  9.9500e-01,  9.9998e-03,  1.0000e+00,  1.0000e-04,
          1.0000e+00,  1.0000e-06,  1.0000e+00],
        [ 9.0930e-01,  9.8007e-01,  1.9999e-02,  1.0000e+00,  2.0000e-04,
          1.0000e+00,  2.0000e-06,  1.0000e+00],
        [ 1.4112e-01,  9.5534e-01,  2.9996e-02,  1.0000e+00,  3.0000e-04,
          1.0000e+00,  3.0000e-06,  1.0000e+00],
        [-7.5680e-01,  9.2106e-01,  3.9989e-02,  9.9999e-01,  4.0000e-04,
          1.0000e+00,  4.0000e-06,  1.0000e+00],
        [-9.5892e-01,  8.7758e-01,  4.9979e-02,  9.9999e-01,  5.0000e-04,
          1.0000e+00,  5.0000e-06,  1.0000e+00]])


In [ ]:
def scaled_dot_product_attention(query, key, value):
    # Calculate the dot products
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(query.size(-1))
    # Apply softmax to get attention weights
    attn_weights = torch.softmax(scores, dim=-1)
    # Multiply attention weights with the values
    output = torch.matmul(attn_weights, value)
    return output, attn_weights

# For demonstration, let's create dummy query, key, and value matrices
query = embeddings.unsqueeze(0)  # Shape: (1, seq_len, d_model)
key = embeddings.unsqueeze(0)
value = embeddings.unsqueeze(0)

# Apply attention
attention_output, attention_weights = scaled_dot_product_attention(query, key, value)

print("Attention Output:\n", attention_output)
print("Attention Weights:\n", attention_weights)


Attention Output:
 tensor([[[-1.0568,  1.3646,  1.3859,  1.0850,  0.7168, -0.0211, -0.3136,
          -1.1182],
         [ 0.4937, -1.4918,  0.6729, -0.3604,  0.2152, -1.0710,  0.2067,
           1.2221],
         [-0.2944, -1.7241, -0.4158,  0.2174,  1.0906, -2.4896, -1.5511,
          -1.8538],
         [ 1.2379, -0.2944,  1.4612, -0.8551, -0.1754, -0.4109,  0.6803,
           0.1745],
         [-0.4633, -0.4630,  2.7504,  1.0397, -0.2349,  0.2810, -1.7601,
           1.6189],
         [ 0.3629,  0.0437,  0.5260, -0.3330,  0.0797,  0.7506,  0.9325,
           0.3509]]], grad_fn=<UnsafeViewBackward0>)
Attention Weights:
 tensor([[[8.6789e-01, 2.9708e-03, 3.2550e-02, 1.4062e-02, 6.6388e-02,
          1.6141e-02],
         [3.8108e-03, 7.2224e-01, 7.0591e-02, 9.5379e-02, 8.3420e-02,
          2.4559e-02],
         [3.3396e-03, 5.6460e-03, 9.8796e-01, 1.3035e-03, 1.5544e-03,
          1.9208e-04],
         [2.2145e-02, 1.1710e-01, 2.0008e-02, 7.2816e-01, 5.2887e-02,
          5.9701e-02]

In [ ]:
class SimpleTransformerEncoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_heads):
        super(SimpleTransformerEncoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.positional_encoding = get_positional_encoding(100, embedding_dim)  # Assuming max seq_len=100
        self.multihead_attn = nn.MultiheadAttention(embed_dim=embedding_dim, num_heads=num_heads)
        self.fc = nn.Linear(embedding_dim, embedding_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.embedding(x)  # (seq_len, batch_size, embedding_dim)
        x = x + self.positional_encoding[:x.size(0), :]
        # PyTorch's MultiheadAttention expects (seq_len, batch, embed_dim)
        attn_output, attn_weights = self.multihead_attn(x, x, x)
        x = attn_output + x  # Residual connection
        x = self.relu(self.fc(x))
        return x, attn_weights

# Initialize the encoder
encoder = SimpleTransformerEncoder(vocab_size=tokenizer.vocab_size, embedding_dim=embedding_size, num_heads=2)

# Prepare input (seq_len, batch_size)
input_tensor = token_ids_tensor.unsqueeze(1)

# Forward pass
encoder_output, encoder_attn_weights = encoder(input_tensor)

print("Encoder Output:\n", encoder_output)
print("Encoder Attention Weights:\n", encoder_attn_weights)


Encoder Output:
 tensor([[[1.9564, 0.0000, 0.0000, 0.0000, 0.1308, 0.0000, 0.0000, 0.5187],
         [2.0188, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5524],
         [2.0234, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5538],
         [1.9659, 0.0000, 0.0000, 0.0000, 0.1009, 0.0000, 0.0000, 0.5204],
         [1.8963, 0.0000, 0.0000, 0.0000, 0.4323, 0.0000, 0.0000, 0.4802],
         [1.8733, 0.0000, 0.0000, 0.0000, 0.5186, 0.0000, 0.0000, 0.4654]],

        [[0.4014, 0.0000, 0.3099, 0.7089, 0.0543, 0.0000, 0.0000, 0.0000],
         [0.4593, 0.0000, 0.4877, 0.7454, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4629, 0.0000, 0.4986, 0.7492, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4088, 0.0000, 0.3287, 0.7174, 0.0258, 0.0000, 0.0000, 0.0000],
         [0.3408, 0.0000, 0.1315, 0.6824, 0.3563, 0.0429, 0.0000, 0.0000],
         [0.3168, 0.0000, 0.0813, 0.6794, 0.4433, 0.0622, 0.0000, 0.0000]],

        [[0.5372, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
    

In [ ]:
# For simplicity, let's assume the translation is a reversed token sequence
translated_tokens = list(reversed(tokens))
translated_ids = tokenizer.convert_tokens_to_ids(translated_tokens)

print("Translated Tokens:", translated_tokens)
print("Translated Token IDs:", translated_ids)

Translated Tokens: ['?', 'you', 'are', 'how', ',', 'hello']
Translated Token IDs: [1029, 2017, 2024, 2129, 1010, 7592]


# Version 2

In [ ]:
from transformers import BertTokenizer

# Initialize tokenizers for source (English) and target (French) languages
source_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
target_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')  # For simplicity, using the same tokenizer

# Example sentence pairs for translation
sentence_pairs = [
    ("Hello, how are you?", "Bonjour, comment ça va?"),
    ("I am fine.", "Je vais bien."),
    ("What is your name?", "Comment tu t'appelles?"),
    ("Good morning.", "Bonjour."),
    ("Thank you.", "Merci.")
]

# Tokenize the sentences
source_sentences = [pair[0] for pair in sentence_pairs]
target_sentences = [pair[1] for pair in sentence_pairs]

source_tokens = [source_tokenizer.tokenize(sentence) for sentence in source_sentences]
target_tokens = [target_tokenizer.tokenize(sentence) for sentence in target_sentences]

source_ids = [source_tokenizer.convert_tokens_to_ids(tokens) for tokens in source_tokens]
target_ids = [target_tokenizer.convert_tokens_to_ids(tokens) for tokens in target_tokens]

# Print tokenized sentences
for i in range(len(sentence_pairs)):
    print(f"Source Sentence: {source_sentences[i]}")
    print(f"Source Tokens: {source_tokens[i]}")
    print(f"Source Token IDs: {source_ids[i]}")
    print(f"Target Sentence: {target_sentences[i]}")
    print(f"Target Tokens: {target_tokens[i]}")
    print(f"Target Token IDs: {target_ids[i]}")
    print("-" * 50)


Source Sentence: Hello, how are you?
Source Tokens: ['hello', ',', 'how', 'are', 'you', '?']
Source Token IDs: [7592, 1010, 2129, 2024, 2017, 1029]
Target Sentence: Bonjour, comment ça va?
Target Tokens: ['bon', '##jou', '##r', ',', 'comment', 'ca', 'va', '?']
Target Token IDs: [14753, 23099, 2099, 1010, 7615, 6187, 12436, 1029]
--------------------------------------------------
Source Sentence: I am fine.
Source Tokens: ['i', 'am', 'fine', '.']
Source Token IDs: [1045, 2572, 2986, 1012]
Target Sentence: Je vais bien.
Target Tokens: ['je', 'va', '##is', 'bien', '.']
Target Token IDs: [15333, 12436, 2483, 29316, 1012]
--------------------------------------------------
Source Sentence: What is your name?
Source Tokens: ['what', 'is', 'your', 'name', '?']
Source Token IDs: [2054, 2003, 2115, 2171, 1029]
Target Sentence: Comment tu t'appelles?
Target Tokens: ['comment', 'tu', 't', "'", 'app', '##elles', '?']
Target Token IDs: [7615, 10722, 1056, 1005, 10439, 22869, 1029]
------------------

In [ ]:
import torch
import torch.nn as nn

# Define embedding size
embedding_size = 16  # Increased size for better representation

# Initialize embedding layers
source_embedding = nn.Embedding(num_embeddings=source_tokenizer.vocab_size, embedding_dim=embedding_size)
target_embedding = nn.Embedding(num_embeddings=target_tokenizer.vocab_size, embedding_dim=embedding_size)

# Example: Embed the first source and target sentence
src_tensor = torch.tensor(source_ids[0]).unsqueeze(1)  # Shape: (seq_len, batch_size)
tgt_tensor = torch.tensor(target_ids[0]).unsqueeze(1)  # Shape: (seq_len, batch_size)

src_embeddings = source_embedding(src_tensor)  # Shape: (seq_len, batch_size, embedding_size)
tgt_embeddings = target_embedding(tgt_tensor)

print("Source Embeddings:\n", src_embeddings)
print("\nTarget Embeddings:\n", tgt_embeddings)


Source Embeddings:
 tensor([[[ 1.0535e+00,  9.1534e-01,  5.0929e-01,  2.2394e-02,  5.2791e-01,
           1.2183e+00, -1.3786e-02,  5.3571e-01, -1.8915e+00,  4.9654e-01,
           2.5310e-01,  5.5887e-01, -1.8654e-02,  1.1799e+00,  7.6429e-01,
           2.1373e-01]],

        [[-4.9310e-01,  1.3174e-01, -3.6768e-01, -1.6604e+00, -1.9670e-01,
          -4.4881e-01, -5.7586e-01, -1.1417e+00, -4.7708e-01, -1.4044e+00,
          -1.1459e+00,  7.2536e-01, -1.7561e+00,  2.8397e-01, -2.9750e-01,
           1.0990e+00]],

        [[ 1.0724e+00, -3.2850e-01,  7.2831e-01, -6.4581e-01,  1.8643e+00,
          -1.2645e+00, -8.4731e-01, -5.4815e-01, -4.3146e-01, -7.7096e-01,
          -2.1870e-01, -2.2842e-01, -3.4257e-01, -1.9705e-01,  6.5944e-01,
           6.3366e-01]],

        [[-7.0318e-01,  2.2166e-03,  2.5875e-01, -1.4474e+00, -1.8114e-01,
           1.1660e+00, -8.3542e-01, -1.1068e+00, -1.2455e-02,  1.5584e+00,
          -1.1140e-01, -9.8011e-01, -8.7823e-01,  1.8507e-01, -2.3282e-01,
  

In [ ]:
import math

def get_positional_encoding(seq_len, d_model):
    pe = torch.zeros(seq_len, d_model)
    for pos in range(seq_len):
        for i in range(0, d_model, 2):
            pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
            if i + 1 < d_model:
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
    return pe

# Example positional encoding for source and target
src_pos_encoding = get_positional_encoding(src_tensor.size(0), embedding_size)
tgt_pos_encoding = get_positional_encoding(tgt_tensor.size(0), embedding_size)

print("Source Positional Encoding:\n", src_pos_encoding)
print("\nTarget Positional Encoding:\n", tgt_pos_encoding)


Source Positional Encoding:
 tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
          1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,
          0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
          1.0000e+00],
        [ 8.4147e-01,  9.5042e-01,  9.9833e-02,  9.9950e-01,  9.9998e-03,
          9.9999e-01,  1.0000e-03,  1.0000e+00,  1.0000e-04,  1.0000e+00,
          1.0000e-05,  1.0000e+00,  1.0000e-06,  1.0000e+00,  1.0000e-07,
          1.0000e+00],
        [ 9.0930e-01,  8.0658e-01,  1.9867e-01,  9.9800e-01,  1.9999e-02,
          9.9998e-01,  2.0000e-03,  1.0000e+00,  2.0000e-04,  1.0000e+00,
          2.0000e-05,  1.0000e+00,  2.0000e-06,  1.0000e+00,  2.0000e-07,
          1.0000e+00],
        [ 1.4112e-01,  5.8275e-01,  2.9552e-01,  9.9550e-01,  2.9996e-02,
          9.9995e-01,  3.0000e-03,  1.0000e+00,  3.0000e-04,  1.0000e+00,
          3.0000e-05,  1.0000e+00,  3.0000e-06,  1.0000e+00,  3.0000e-07,
          1.00

In [ ]:
import torch.optim as optim

# Define the Transformer model
class SimpleTransformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, embedding_size, num_heads, num_encoder_layers, num_decoder_layers, dim_feedforward=512, dropout=0.1):
        super(SimpleTransformer, self).__init__()
        self.src_embedding = nn.Embedding(src_vocab_size, embedding_size)
        self.tgt_embedding = nn.Embedding(tgt_vocab_size, embedding_size)
        self.pos_encoder = nn.Embedding(1000, embedding_size)  # Assuming max seq_len=1000
        self.pos_decoder = nn.Embedding(1000, embedding_size)
        self.transformer = nn.Transformer(d_model=embedding_size, nhead=num_heads,
                                          num_encoder_layers=num_encoder_layers,
                                          num_decoder_layers=num_decoder_layers,
                                          dim_feedforward=dim_feedforward,
                                          dropout=dropout)
        self.fc_out = nn.Linear(embedding_size, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, tgt):
        src_seq_length = src.size(0)
        tgt_seq_length = tgt.size(0)

        # Create position tensors
        src_positions = torch.arange(0, src_seq_length).unsqueeze(1).to(src.device)
        tgt_positions = torch.arange(0, tgt_seq_length).unsqueeze(1).to(tgt.device)

        # Embed and add positional encoding
        src_emb = self.src_embedding(src) + self.pos_encoder(src_positions)
        tgt_emb = self.tgt_embedding(tgt) + self.pos_decoder(tgt_positions)

        src_emb = self.dropout(src_emb)
        tgt_emb = self.dropout(tgt_emb)

        # Generate masks
        tgt_mask = self.transformer.generate_square_subsequent_mask(tgt_seq_length).to(src.device)

        # Pass through Transformer
        output = self.transformer(src_emb, tgt_emb, tgt_mask=tgt_mask)

        # Output layer
        output = self.fc_out(output)
        return output

# Initialize the Transformer
transformer_model = SimpleTransformer(
    src_vocab_size=source_tokenizer.vocab_size,
    tgt_vocab_size=target_tokenizer.vocab_size,
    embedding_size=embedding_size,
    num_heads=4,
    num_encoder_layers=2,
    num_decoder_layers=2
)

print(transformer_model)


SimpleTransformer(
  (src_embedding): Embedding(30522, 16)
  (tgt_embedding): Embedding(30522, 16)
  (pos_encoder): Embedding(1000, 16)
  (pos_decoder): Embedding(1000, 16)
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-1): 2 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=16, out_features=16, bias=True)
          )
          (linear1): Linear(in_features=16, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=512, out_features=16, bias=True)
          (norm1): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
      (norm): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
    )

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [ ]:
# Define hyperparameters
num_epochs = 1500
learning_rate = 0.001

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(transformer_model.parameters(), lr=learning_rate)

# Move model to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transformer_model = transformer_model.to(device)

# Prepare data
# For simplicity, we'll use the entire dataset as a batch
def prepare_batch(sentence_pairs):
    src_batch = []
    tgt_batch = []
    for src, tgt in sentence_pairs:
        src_ids = source_tokenizer.convert_tokens_to_ids(source_tokenizer.tokenize(src))
        tgt_ids = target_tokenizer.convert_tokens_to_ids(target_tokenizer.tokenize(tgt))
        src_batch.append(src_ids)
        tgt_batch.append(tgt_ids)

    # Pad sequences
    src_padded = nn.utils.rnn.pad_sequence([torch.tensor(seq) for seq in src_batch], padding_value=source_tokenizer.pad_token_id)
    tgt_padded = nn.utils.rnn.pad_sequence([torch.tensor(seq) for seq in tgt_batch], padding_value=target_tokenizer.pad_token_id)

    return src_padded.to(device), tgt_padded.to(device)

src_data, tgt_data = prepare_batch(sentence_pairs)


In [ ]:
for epoch in range(num_epochs):
    transformer_model.train()

    optimizer.zero_grad()

    # Prepare input and target
    src = src_data  # Shape: (src_seq_len, batch_size)
    tgt = tgt_data[:-1, :]  # Input to the decoder (excluding the last token)
    tgt_y = tgt_data[1:, :]  # Target output (excluding the first token)

    # Forward pass
    output = transformer_model(src, tgt)
    # Reshape output and target for loss calculation
    output = output.view(-1, output.size(-1))
    tgt_y = tgt_y.contiguous().view(-1)

    # Compute loss
    loss = criterion(output, tgt_y)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if (epoch+1) % 20 == 0 or epoch == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


Epoch [1/1500], Loss: 0.1222
Epoch [20/1500], Loss: 0.0846
Epoch [40/1500], Loss: 0.0461
Epoch [60/1500], Loss: 0.0388
Epoch [80/1500], Loss: 0.0249
Epoch [100/1500], Loss: 0.0195
Epoch [120/1500], Loss: 0.0203
Epoch [140/1500], Loss: 0.0121
Epoch [160/1500], Loss: 0.0183
Epoch [180/1500], Loss: 0.0137
Epoch [200/1500], Loss: 0.0106
Epoch [220/1500], Loss: 0.0083
Epoch [240/1500], Loss: 0.0079
Epoch [260/1500], Loss: 0.0077
Epoch [280/1500], Loss: 0.0063
Epoch [300/1500], Loss: 0.0057
Epoch [320/1500], Loss: 0.0415
Epoch [340/1500], Loss: 0.0078
Epoch [360/1500], Loss: 0.0063
Epoch [380/1500], Loss: 0.0055
Epoch [400/1500], Loss: 0.0037
Epoch [420/1500], Loss: 0.0189
Epoch [440/1500], Loss: 0.0079
Epoch [460/1500], Loss: 0.0038
Epoch [480/1500], Loss: 0.0034
Epoch [500/1500], Loss: 0.0040
Epoch [520/1500], Loss: 0.0087
Epoch [540/1500], Loss: 0.0028
Epoch [560/1500], Loss: 0.0040
Epoch [580/1500], Loss: 0.0028
Epoch [600/1500], Loss: 0.0031
Epoch [620/1500], Loss: 0.0029
Epoch [640/150

In [ ]:
# Function to generate translation
def translate_sentence(model, sentence, source_tokenizer, target_tokenizer, max_length=20):
    model.eval()

    # Tokenize and convert to IDs
    src_tokens = source_tokenizer.tokenize(sentence)
    src_ids = source_tokenizer.convert_tokens_to_ids(src_tokens)
    src_tensor = torch.tensor(src_ids).unsqueeze(1).to(device)  # Shape: (src_seq_len, 1)

    # Initialize target with <sos> token if available, else use first token
    # For simplicity, we'll use the first token of the target language as start
    tgt_ids = [target_tokenizer.cls_token_id] if target_tokenizer.cls_token_id is not None else [target_tokenizer.pad_token_id]
    tgt_tensor = torch.tensor(tgt_ids).unsqueeze(1).to(device)

    for _ in range(max_length):
        # Forward pass
        output = model(src_tensor, tgt_tensor)
        # Get the last token's prediction
        last_token_logits = output[-1, 0, :]
        _, next_token = torch.max(last_token_logits, dim=0)
        next_token = next_token.item()

        # Append the predicted token
        tgt_tensor = torch.cat([tgt_tensor, torch.tensor([[next_token]]).to(device)], dim=0)

        # If end token is predicted, stop
        if next_token == target_tokenizer.sep_token_id or next_token == target_tokenizer.pad_token_id:
            break

    # Convert tokens to words
    translated_ids = tgt_tensor.squeeze().tolist()
    translated_tokens = target_tokenizer.convert_ids_to_tokens(translated_ids)

    # Remove special tokens if present
    translated_tokens = [token for token in translated_tokens if token not in [target_tokenizer.cls_token, target_tokenizer.sep_token, target_tokenizer.pad_token]]

    # Detokenize
    translated_sentence = target_tokenizer.convert_tokens_to_string(translated_tokens)

    return translated_sentence

# Example sentences for translation
new_sentences = [
    "Good night.",
    "How old are you?",
    "See you later."
]

for sentence in new_sentences:
    translation = translate_sentence(transformer_model, sentence, source_tokenizer, target_tokenizer)
    print(f"English: {sentence}")
    print(f"French: {translation}")
    print("-" * 50)


English: Good night.
French: ##jour .
--------------------------------------------------
English: How old are you?
French: ##jour , comment ca va ?
--------------------------------------------------
English: See you later.
French: ##jour .
--------------------------------------------------
